In [5]:

pip install mysql-connector-python pandas 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# === 1. Load files to Dataframes ===
agri_df = pd.read_csv(r"..\data\agriculture_projects.csv")
carbon_df = pd.read_json(r"..\data\carbon_monitoring.json", lines=True)
fund_df = pd.read_csv(r"..\data\funding_data.csv")

# === 2. Clean and manipulate data ===

# creating a new column called carbon_gain by subtracting soil_carbon_baseline from soil_carbon_current times area_ha,
# multiplying with 3.67 which converts tons of carbon to tons of CO₂ equivalent
agri_df["carbon_gain_tCO2e"] = (
    (agri_df["soil_carbon_current"] - agri_df["soil_carbon_baseline"])
    * agri_df["area_ha"] * 3.67
).round(2)

carbon_df["credits_verified"] = carbon_df["credits_verified"].where(carbon_df["credits_verified"] > 0, 0).astype(int)
fund_df["disbursed_date"] = pd.to_datetime(fund_df["disbursed_date"], format="%m/%d/%Y").dt.strftime("%Y-%m-%d")


conn = None
try:
    # === 3. Create a connection with MySQL ===
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="organic_farming_db"
    )
    cursor = conn.cursor()

    print("Connected to MySQL database")

    # === 4. Delete all records from the table to avoid duplicate records ===
    cursor.execute("DELETE FROM funding_data;")
    cursor.execute("DELETE FROM carbon_monitoring;")
    cursor.execute("DELETE FROM agriculture_projects;")
    print("All records removed from db")
    
    project_ids = {}
    # === 5. Insert data into agriculture_projects ===
    for _, row in agri_df.iterrows():
        cursor.execute("""
            INSERT INTO agriculture_projects (
                region, crop_type, area_ha,
                soil_carbon_baseline, soil_carbon_current,
                farmers, start_year, carbon_gain_tCO2e
            ) VALUES (%s,%s,%s,%s,%s,%s,%s, %s)
        """, (
            row.region, row.crop_type,
            int(row.area_ha), float(row.soil_carbon_baseline),
            float(row.soil_carbon_current), int(row.farmers), int(row.start_year),
            row.carbon_gain_tCO2e
        ))
        last_project_id = cursor.lastrowid
        # print(f"Last Id: {last_project_id}")
        if last_project_id > 0:
            project_ids[row.project_id] = last_project_id
    conn.commit()
    print("agriculture_projects table updated")

    # === 6. Insert data into carbon_monitoring ===
    for _, row in carbon_df.iterrows():
        cursor.execute("""
            INSERT INTO carbon_monitoring (
                project_id, mrv_status, verification_date,
                credits_verified, verifier, remarks,
                last_update, monitoring_round
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """, (
            project_ids[row.project_id], row.mrv_status, row.verification_date,
            row.credits_verified, row.verifier, row.remarks,
            row.last_update, row.monitoring_round
        ))
    conn.commit()
    print("carbon_monitoring table updated")

    # === 7. Insert data into funding_data ===
    for _, row in fund_df.iterrows():
        
        cursor.execute("""
            INSERT INTO funding_data (
                project_id, funding_source, amount_usd,
                disbursed_date, purpose, contract_id,
                disbursed_by, currency
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """, (
            project_ids[row.project_id], row.funding_source, float(row.amount_usd),
            row.disbursed_date, row.purpose, row.contract_id,
            row.disbursed_by, row.currency
        ))
    conn.commit()
    print("funding_data table updated")

except Error as e:
    print(f"error {e}")
finally:
    # === 8. Close all open connections ===
    if conn and conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection closed")


Connected to MySQL database
agriculture_projects table updated
carbon_monitoring table updated
funding_data table updated
MySQL connection closed
